Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Tarea 5 </h2>
    <h1> SVM </h1>
    <p>
        Profesor Marcelo Mendoza<br>
        Segundo Semestre 2023<br> 
        Fecha de entrega: 3 de noviembre
    </p>
    <br>
</center>

<br>


## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas.

**IMPORTANTE**:
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas. Es decir, si tienes un código perfecto pero este no es explicado o no se responden preguntas asociadas a este, no se tendrá el puntaje completo.
- El notebook debe tener todas las celdas de código ejecutadas. Cualquier notebook que no las tenga no podrá ser corregido.
- El carácter de esta tarea es **INDIVIDUAL**. Cualquier instancia de copia resultará en un 1,1 como nota de curso.
- En el caso de que se encuentren con problemas al correr celdas por el tamaño del dataset, esta permitido trabajar con una muestra representativa de este, siempre explicitando y justificando sus deciciones.

---


## Librerías

In [1]:
##Importa acá las librerias que vayas a utilizar
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from lime.lime_text import LimeTextExplainer

c:\Users\Gabi\Documents\U-2023-2\IIC2433\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Contexto:

Se tiene un dataset con información de distintas noticias, entre las cuales hay noticias falsas y verdaderas. El objetivo de esta tarea es predecir si una noticia es falsa o verdadera, utilizando distintos modelos de clasificación.. Para esto, primero se debe hacer un análisis exploratorio de los datos para decidir qué variables son relevantes para el problema. Luego, se debe vectorizar el texto de loas noticias para poder utilizarlo en un modelo de clasificación (en esta tarea usaremos SBert y TF-IDF). Después se debe entrenar un modelo de clasificación SVM que prediga si una noticia es falsa o verdadera. Finalmente, se debe evaluar el modelo y compararlo con otros modelos de clasificación.

# Objetivos de la tarea:

- Realizar un análisis exploratorio de datos para determinar las variables pertinentes para el problema en cuestión, identificando características clave y patrones en los datos que podrían afectar la clasificación de las noticias.
- Dominar la técnica de vectorización de texto para facilitar su aplicación en un modelo de clasificación, y comprender la interpretación de los resultados obtenidos.
- Entender el funcionamiento de un modelo de clasificación SVM y sus hiperparámetros.
- Entrenar un modelo de clasificación SVM capaz de predecir si una noticia es falsa o verdadera.
- Evaluar el desempeño del modelo y analizar en profundidad los resultados obtenidos.

# Parte 1: Carga y Preprocesamiento (10 puntos)

## 1.1 Carga de datos (1 punto)

Para esta tarea deberás trabajar con el dataset que está en Canvas, 'news.csv'. Este dataset contiene información de distintas noticias, entre las cuales hay noticias falsas y verdaderas.

In [2]:
data = pd.read_csv('news.csv')
data.head()

,title,text,subject,date,authenticity
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake


## 1.2 Descripcion del Dataset (5 puntos)

In [3]:
len(data.iloc[0]['text'])

2893

¿Qué representa cada feature en el dataset entregado? Refiérete a su tipo de dato, detallando cómo trabajar con los datos no númericos (2 puntos)

RESPUESTA:

- title : Título de la noticia
- text : Cuerpo de la noticia
- subject : Tema de la noticia
- date : Fecha de publicación de la noticia
- authenticity : Variable objetivo, indica si la noticia es falsa o verdadera

¿Qué columnas o features crees son relevantes para el problema? ¿Por qué? (3 puntos)

RESPUESTA:  
El cuerpo, ya que se cuenta con una mayor cantidad de palabras que los títulos. Además, el título puede ser engañoso, por lo que el cuerpo de la noticia es más relevante para determinar si es falsa o verdadera. El tema tambien considero que puede ser relevante ya que puede haber temas que sean más propensos a tener noticias falsas o verdaderas, en el caso de este dataset, solo se cuenta con 2 tipos por lo que no creo que sea tan relevante.

## 1.3 Datos nulos (2 puntos)

Analiza la presencia de valores nulos en el conjunto de datos y cómo se distribuyen en las distintas columnas. Después, toma una decisión acerca del tratamiento adecuado para el dataframe con respecto a los valores nulos. Justifica tu decisión.

RESPUESTA:


In [4]:
data.isna().sum()

title           0
text            0
subject         0
date            0
authenticity    0
dtype: int64

In [5]:
data.nunique()

title           9954
text            9962
subject            2
date             787
authenticity       2
dtype: int64

Se puede ver que no hay valores nulos en el dataset, por lo que no es necesario realizar un tratamiento de valores nulos.

## 1.4 Manejo del Dataset (2 puntos)

- Elimina las columnas que no sean relevantes para el entrenamiento del modelo (1 pts.)
- Haz que los textos estén en un formato óptimo para ser procesados por los modelos de clasificación (0.5 pts.) Justifica tu decisión (0.5 pts.)



RESPUESTA:

In [6]:
y = data['authenticity']
X = data.drop(['authenticity'], axis=1)

Para que los textos esten en un formato optimo, se puede utilizar la libreria NLTK para eliminar stopwords y lematizar las palabras. Además, se puede utilizar la libreria re para eliminar caracteres especiales y numeros.

In [7]:
import nltk
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gabi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gabi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [9]:
stop_words = set(stopwords.words("english"))

In [10]:
def no_stop_words(text):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return " ".join(filtered_sentence)

In [11]:
X_ = X.copy()
X_['title'] = X_['title'].apply(no_stop_words)
X_['text'] = X['text'].apply(no_stop_words)
X_ = X_.drop(['date', 'subject'], axis=1)

# Parte 2: Vectorización (14 puntos)

# SBert

Para esta parte, deben utilizar SBert para vectorizar los textos de las noticias. Utilicen el modelo pre-entrenado de SBert llamado SentenceTransformer. Específicamente, deben usar el modelo 'paraphrase-MiniLM-L6-v2' para obtener las representaciones vectoriales de las oraciones.

### 2.1) Vectorización con SBert (3 puntos)

In [13]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [14]:
sentences = X_['text'].to_list()

#Sentences are encoded by calling model.encode()
SBert_texts = model.encode(sentences)

### 2.2) Análisis teórico de SBert (4 puntos)
Responde las siguientes preguntas: ¿Qué es SBert? ¿Cómo funciona? ¿Qué ventajas y desventajas tiene sobre otros modelos de vectorización de texto?

RESPUESTA:

SBert, Sentence BERT, es una técnica de vectorización de texto que utiliza un modelo pre-entrenado para obtener representaciones vectoriales de oraciones. Para esto, se utiliza un modelo pre-entrenado de BERT (Bidirectional Encoder Representations from Transformers), que es un modelo de lenguaje que utiliza una arquitectura de Transformer. Con este se obtienen representaciones vectoriales de oraciones, en vez de palabras. Primero se utiliza el modelo pre-entrenado BERT para las representaciones vectoriales de palabras y luego se utiliza un modelo de pooling promedio para obtener una representación vectorial de la oración, promediando los vectores de las palabras de la oración.

Ventajas:
- Mejora la captura de la semántica de las oraciones.
- Reduce la necesidad de entrenamiento ya que utiliza un modelo pre-entrenado.

Desventajas:
- Al utilizar un modelo basado en transformers, puede ser computacionalmente costoso.
- Necesita Ajuste y Finetuning para obtener mejores resultados.

fuentes:   
https://arxiv.org/abs/1908.10084  
https://medium.com/mlearning-ai/semantic-search-with-s-bert-is-all-you-need-951bc710e160  
https://towardsdatascience.com/an-intuitive-explanation-of-sentence-bert-1984d144a868


# TF-IDF

Para esta parte, deben utilizar TF-IDF para vectorizar los textos de las noticias. Utilicen la clase TfidfVectorizer de la librería sklearn.


Tip: limita la cantidad de features.

### 2.3) Vectorización con TF-IDF (3 puntos)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

max_features = 100
vectorizer = TfidfVectorizer(max_features=max_features)

TfIdf_texts = vectorizer.fit_transform(sentences)

### 2.2) Análisis teórico de TF-IDF (4 puntos)
Responde las siguientes preguntas: ¿Qué es TF-IDF? ¿Cómo funciona? ¿Qué ventajas y desventajas tiene sobre otros modelos de vectorización de texto?

RESPUESTA:

Literalmente significa *Term frecuency - Inverse document fercuency*, lo que quiere decir la frecuencia de ocurrencia del término en el documento. Es una medida estadística que se utiliza para evaluar la importancia de una palabra en un documento. Se considera que a mayor frecuencia de una palabra, más importancia tiene en el documento. 

La ventaja de este modelo es que es simple de implementar y es muy eficiente computacionalmente. Una desventaja es que no considera la semántica de las palabras, por lo que puede haber palabras que tengan un alto TF-IDF pero que no sean relevantes para el documento, por eso es importante quitar las 'stop words' antes de realizar una analisis.

En cuanto a otros modelos de vectorización, se puede decir que es más simple de implementar que SBert, pero no considera la semántica de las palabras, lo que si realiza SBert.

fuentes:  
https://www.geeksforgeeks.org/understanding-tf-idf-term-frequency-inverse-document-frequency/  
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

# Parte 3: SVM (36 puntos)

### 3.1) Preguntas teóricas (10 puntos)
1. ¿Qué es SVM? ¿Cómo funciona? ¿En qué casos es útil? ¿En cuáles no? (4 ptos.)  
    Una *Support-Vector Machine*, o maquina de vectores de soporte, es un tipo de algoritmo de aprendizaje supervisado que puede ser utilizado para clasificación o regresión. La idea es generar un hiperplano separador que maximice la distancia entre los puntos de datos de las clases (para clasificación).

    Busca el hiperplano maximizando el margen entre las clases. Los puntos mas cercanos al hiperplano son los vectores de soporte, y en base a ellos se clasifican nuevos puntos.

    Son útiles cuando el conjunto de datos tiene múltiples caracteristicas y la separación entre clases es clara. No son útiles cuando el conjunto de datos tiene muchas caracteristicas y la separación entre clases no es clara.
    
3. ¿Qué es un kernel? ¿Qué tipos de kernels existen? ¿Cuál es la diferencia entre ellos? (3 ptos.)  
    Un kernel en este contexto es una función que toma datos de entrada de baja dimensionalidad y los transforma en datos de alta dimensionalidad. Existen distintos tipos de kernels, como el kernel lineal, polinomial, gaussiano, etc. La diferencia entre ellos es la función que utilizan para transformar los datos de entrada. Cada kernel realiza una transformación diferente sobre los datos.

4. ¿Qué indica el parámetro C? ¿Qué sucede si C es muy grande? ¿Y si es muy pequeño? (3 ptos.)  
    Es el parametro de regularización, que controla el equilibrio entre maximizar el margen y minimizar la clasificación incorrecta. Si C es muy grande, se penaliza mucho la clasificación incorrecta, por lo que se puede generar un sobreajuste. Si C es muy pequeño, se penaliza poco la clasificación incorrecta, por lo que se puede generar un subajuste.

fuentes:  
https://monkeylearn.com/blog/introduction-to-support-vector-machines-svm/  
https://scikit-learn.org/stable/modules/svm.html  
https://www.geeksforgeeks.org/introduction-to-support-vector-machines-svm/

### 3.2) SVM con vectorización SBert (10 puntos) 
Para esta parte deben entrenar un modelo SVM con las representaciones vectoriales obtenidas con SBert. Deben utilizar la clase SVC de la librería sklearn. Para esto:
- Deben dejar los datos en un formato adecuado para ser procesados por el modelo (1 pto.)
- Dividir el dataset en train y test (1 pto.)
- Entrenar el modelo SVM con los datos de train (2 ptos.) modificando los hiperparámetros kernel y C (4 ptos.)
- Evaluar el modelo con los datos de test y comentar brevemente los resultados obtenidos (2 ptos.).

In [19]:
X_train, X_test, y_train, y_test = train_test_split(SBert_texts, y, test_size=0.2, random_state=42)

In [21]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
C = [0.1, 1, 10] #, 100, 1000]
degrees = [2, 3, 4, 5, 6]
clfs = {}

for kernel in kernels:
    for c in C:
        print(f"Kernel: {kernel}, C: {c}")
        if kernel == 'poly':
            for degree in degrees:
                print(f"Degree: {degree}")
                clf = SVC(kernel=kernel, C=c, degree=degree)
                clf.fit(X_train, y_train)
                clfs[(kernel, c, degree)] = clf
                accuracy = clf.score(X_test, y_test)
                print(f'accuracy: {accuracy}')
        else:
            clf = SVC(kernel=kernel, C=c)
            clf.fit(X_train, y_train)
            clfs[(kernel, c, 0)] = clf
            accuracy = clf.score(X_test, y_test)
            print(f'accuracy: {accuracy}')

Kernel: linear, C: 0.1
accuracy: 0.9915
Kernel: linear, C: 1
accuracy: 0.9895
Kernel: linear, C: 10
accuracy: 0.985
Kernel: poly, C: 0.1
Degree: 2
accuracy: 0.965
Degree: 3
accuracy: 0.942
Degree: 4
accuracy: 0.9115
Degree: 5
accuracy: 0.8645
Degree: 6
accuracy: 0.827
Kernel: poly, C: 1
Degree: 2
accuracy: 0.989
Degree: 3
accuracy: 0.985
Degree: 4
accuracy: 0.981
Degree: 5
accuracy: 0.966
Degree: 6
accuracy: 0.9455
Kernel: poly, C: 10
Degree: 2
accuracy: 0.9885
Degree: 3
accuracy: 0.984
Degree: 4
accuracy: 0.982
Degree: 5
accuracy: 0.977
Degree: 6
accuracy: 0.9655
Kernel: rbf, C: 0.1
accuracy: 0.9635
Kernel: rbf, C: 1
accuracy: 0.9875
Kernel: rbf, C: 10
accuracy: 0.9895
Kernel: sigmoid, C: 0.1
accuracy: 0.9685
Kernel: sigmoid, C: 1
accuracy: 0.96
Kernel: sigmoid, C: 10
accuracy: 0.936


Luego de analizar el outout completo, se puede notar que la mayor accuracy, fue obtenida por el kernel lineal con C=1, con un valor de 0.9895. Esto me sorprendio ya que asumí que los datos de texto no se ajustarian correctamente a un separador lineal.

In [23]:
clfs.keys()

dict_keys([('linear', 0.1, 0), ('linear', 1, 0), ('linear', 10, 0), ('poly', 0.1, 2), ('poly', 0.1, 3), ('poly', 0.1, 4), ('poly', 0.1, 5), ('poly', 0.1, 6), ('poly', 1, 2), ('poly', 1, 3), ('poly', 1, 4), ('poly', 1, 5), ('poly', 1, 6), ('poly', 10, 2), ('poly', 10, 3), ('poly', 10, 4), ('poly', 10, 5), ('poly', 10, 6), ('rbf', 0.1, 0), ('rbf', 1, 0), ('rbf', 10, 0), ('sigmoid', 0.1, 0), ('sigmoid', 1, 0), ('sigmoid', 10, 0)])

### 3.3) SVM con vectorización TF-IDF (10 puntos) 
Para esta parte deben entrenar un modelo SVM con las representaciones vectoriales obtenidas con TF-IDF. Deben utilizar la clase SVC de la librería sklearn. Para esto:
- Deben dejar los datos en un formato adecuado para ser procesados por el modelo (1 pto.)
- Dividir el dataset en train y test (1 pto.)
- Entrenar el modelo SVM con los datos de train (2 ptos.) modificando los hiperparámetros kernel y C (4 ptos.)
- Evaluar el modelo con los datos de test y comentar brevemente los resultados obtenidos (2 ptos.).

### 3.4) Análisis de resultados (6 puntos)
- ¿Qué vectorización obtuvo mejores resultados, SBert o TF-IDF? ¿Por qué? (3 ptos.)
- ¿Qué hiperparámetros obtuvieron mejores resultados para cada vectorización? (3 ptos.)

# Parte 4 (Bonus): LIME explainer (5 puntos)

A continuación haz un análisis de los resultados obtenidos con el modelo SVM con vectorización SBert utilizando LIME explainer. Para esto, debes seguir los siguientes pasos:
- Instalar la librería Lime
- Elegir un ejemplo de test
- Utilizar el explainer de Lime para explicar la predicción del modelo en ese ejemplo (2.5 pts.)
- Analizar los resultados obtenidos y comentar brevemente (2.5 pts.)